In [ ]:
#| default_exp process

# Process

> API details.

In [ ]:
#| export
from avcv._imports import *

import mmcv
import pandas
import matplotlib


<string>:1: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
<string>:1: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
<string>:1: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [ ]:
#| export
def multi_thread(fn, array_inputs, max_workers=None, 
                 desc="", unit="Samples", 
                 verbose=True, pbar_iterval=10):

    def _wraper(x):
        i, input = x
        return {i: fn(input)}

    array_inputs = [(i, _) for i, _ in enumerate(array_inputs)]
    if verbose:
        logger.info(desc)
        progress_bar = mmcv.ProgressBar(len(array_inputs))#tqdm(total=len(array_inputs))
    outputs = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for i, result in enumerate(executor.map(_wraper, array_inputs)):
            outputs.update(result)
            if verbose and i%pbar_iterval==0:
                progress_bar.update(pbar_iterval)
    # if verbose:
        # logger.info('multi_thread {}, {}', fn.__name__, desc)
    outputs = list(outputs.values())
    return outputs




In [ ]:
#| hide
show_doc(multi_thread)
def add(x):return x+1
multi_thread(add, range(5), verbose=1)

2022-11-10 04:04:27.315 | INFO     | __main__:multi_thread:12 - 


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/5, 349.7 task/s, elapsed: 0s, ETA:     0s

[1, 2, 3, 4, 5]

In [ ]:
#|export
def multi_process(f, inputs, max_workers=8, desc='',
               unit='Samples', verbose=True, pbar_iterval=10):
    if verbose:
        
        logger.info('Multi processing {} | Num samples: {}', f.__name__, len(inputs))
        pbar = mmcv.ProgressBar(len(inputs))
        
    with Pool(max_workers) as p:
        it = p.imap(f, inputs)
        total = len(inputs)
        # return list(tqdm(it, total=total))
        return_list = []
        for i, ret in enumerate(it):
            return_list.append(ret)
            if i % pbar_iterval == 0 and verbose:
                pbar.update(pbar_iterval)
    return return_list



In [ ]:
#|hide
show_doc(multi_process)
multi_process(add, range(5), verbose=1)

2022-11-10 04:04:28.550 | INFO     | __main__:multi_process:6 - Multi processing add | Num samples: 5


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/5, 149.1 task/s, elapsed: 0s, ETA:     0s

[1, 2, 3, 4, 5]

In [ ]:
#| hide

def f(x):
    time.sleep(np.random.uniform(0.1, .5))
multi_process(f, range(20), );

2022-11-10 04:04:28.774 | INFO     | __main__:multi_process:6 - Multi processing f | Num samples: 20


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 20/20, 22.4 task/s, elapsed: 1s, ETA:     0s

## Example

In [ ]:
show_doc(multi_thread)


def f(i):
    time.sleep(0.1)
    return i
timer = mmcv.Timer()

multi_thread(f, range(6), max_workers=2)
assert timer.since_start() < 2
print(f'Since start: {timer.since_start():0.2f} s')
multi_process(f, range(10))

2022-11-10 04:04:29.807 | INFO     | __main__:multi_thread:12 - 


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/6, 81.7 task/s, elapsed: 0s, ETA:     0s

2022-11-10 04:04:30.146 | INFO     | __main__:multi_process:6 - Multi processing f | Num samples: 10


Since start: 0.34 s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 62.6 task/s, elapsed: 0s, ETA:     0s

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
!nbdev_export